In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pathlib
import pickle

sys.path.append('..')

In [3]:
import pandas as pd

In [4]:
from pals.pimp_tools import get_authentication_token, get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.feature_extraction import DataSource
from pals.pathway_analysis import PALS
from pals.common import *

2020-01-06 11:22:52.112 | WARNING  | pals.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2020-01-06 11:22:52.114 | WARNING  | pals.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2020-01-06 11:22:53.185 | INFO     | pals.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# CRC Data Analysis 

In [5]:
crc_table = pd.read_csv('crc_with_kegg.csv')

In [6]:
crc_table.head()

,Unnamed: 0,Metabolites,Kegg Id,Patient group:CRC,Patient group:CRC.1,Patient group:CRC.2,Patient group:CRC.3,Patient group:CRC.4,Patient group:CRC.5,Patient group:CRC.6,...,Patient group:Polyp.66,Patient group:Polyp.67,Patient group:Polyp.68,Patient group:Polyp.69,Patient group:Polyp.70,Patient group:Polyp.71,Patient group:Polyp.72,Patient group:Polyp.73,Patient group:Polyp.74,Patient group:Polyp.75
0,1,1-Methyladenosine,C02494,390953.9725,567984.4359,558842.7545,476949.0000,398317.0000,411224.0000,412250.6387,...,390493.7447,375952.1076,399195.9585,1.018448e+06,503210.6702,311604.8089,514713.0726,407366.5548,463101.8944,510398.3294
1,2,1-Methylhistamine,C05127,34627.3365,52845.9284,31507.3427,29397.0000,38877.0000,25493.0000,44478.1854,...,18266.0890,30752.9780,41086.0892,2.094303e+04,27630.5961,30531.1781,24293.0652,29915.2947,32606.5463,29862.9664
2,3,2-Aminoadipate,C00956,141257.3640,528024.6865,188272.3016,107076.0918,158299.0710,136600.4976,235936.4772,...,149966.9484,188581.2257,153364.1518,2.666709e+06,228074.4199,132828.4060,215838.8101,179582.5152,227973.0218,216277.0586
3,4,2-Deoxyuridine,C00526,13115.8131,14327.6960,12756.8649,10539.0000,11689.0000,12425.0000,15692.6312,...,11790.1935,11041.2359,13050.2462,1.094457e+04,13652.0542,9250.8437,14597.4711,14137.2620,13281.2552,12653.4161
4,5,4-Pyridoxic acid,C00847,333815.0605,402460.7838,389636.3191,373058.7101,323706.9435,358042.2763,355881.3356,...,379298.6241,356057.1511,412505.9735,4.260027e+05,358592.6192,314369.6151,327164.9675,361262.1060,376591.8864,368266.5694


#### Reset index position and name

In [7]:
crc_table.set_index('Unnamed: 0', inplace=True)
crc_table.index.name = 'row_id'

#### Get the annotation dataframe using the index and the KEGG IDs

In [8]:
crc_annotation_df = crc_table[['Kegg Id']]
crc_annotation_df.columns = ['entity_id']
crc_annotation_df.head()

,entity_id
row_id,
1,C02494
2,C05127
3,C00956
4,C00526
5,C00847


#### Drop columns not required by the intensity DF

In [9]:
crc_int_df = crc_table.drop(columns=['Metabolites', 'Kegg Id'])
crc_int_df.head()

,Patient group:CRC,Patient group:CRC.1,Patient group:CRC.2,Patient group:CRC.3,Patient group:CRC.4,Patient group:CRC.5,Patient group:CRC.6,Patient group:CRC.7,Patient group:CRC.8,Patient group:CRC.9,...,Patient group:Polyp.66,Patient group:Polyp.67,Patient group:Polyp.68,Patient group:Polyp.69,Patient group:Polyp.70,Patient group:Polyp.71,Patient group:Polyp.72,Patient group:Polyp.73,Patient group:Polyp.74,Patient group:Polyp.75
row_id,,,,,,,,,,,,,,,,,,,,,
1,390953.9725,567984.4359,558842.7545,476949.0000,398317.0000,411224.0000,412250.6387,414501.1114,416166.0000,384549.0000,...,390493.7447,375952.1076,399195.9585,1.018448e+06,503210.6702,311604.8089,514713.0726,407366.5548,463101.8944,510398.3294
2,34627.3365,52845.9284,31507.3427,29397.0000,38877.0000,25493.0000,44478.1854,27448.7350,34757.0000,26494.0000,...,18266.0890,30752.9780,41086.0892,2.094303e+04,27630.5961,30531.1781,24293.0652,29915.2947,32606.5463,29862.9664
3,141257.3640,528024.6865,188272.3016,107076.0918,158299.0710,136600.4976,235936.4772,419826.6337,351043.7015,124681.8762,...,149966.9484,188581.2257,153364.1518,2.666709e+06,228074.4199,132828.4060,215838.8101,179582.5152,227973.0218,216277.0586
4,13115.8131,14327.6960,12756.8649,10539.0000,11689.0000,12425.0000,15692.6312,15743.5808,8418.0000,9693.0000,...,11790.1935,11041.2359,13050.2462,1.094457e+04,13652.0542,9250.8437,14597.4711,14137.2620,13281.2552,12653.4161
5,333815.0605,402460.7838,389636.3191,373058.7101,323706.9435,358042.2763,355881.3356,347876.8878,347660.2188,354733.2473,...,379298.6241,356057.1511,412505.9735,4.260027e+05,358592.6192,314369.6151,327164.9675,361262.1060,376591.8864,368266.5694


### Make group lists for the experimental design

In [10]:
columns = crc_int_df.columns
crc = []
healthy = []
polyp = []

for c in columns:
    if 'CRC' in c:
        crc.append(c)
    elif 'Healthy' in c:
        healthy.append(c)
    elif 'Polyp' in c:
        polyp.append(c)
        
crc_experimental_design = {
    'comparisons': [
        {'case': 'CRC', 'control': 'healthy', 'name': 'crc/healthy'},
        {'case': 'polyp', 'control': 'healthy', 'name': 'polyp/healthy'}
    ]
}

In [11]:
crc_experimental_design['groups'] = {'healthy':healthy, 'CRC':crc, 'polyp': polyp}

In [12]:
crc_ds = DataSource(crc_int_df, crc_annotation_df, crc_experimental_design, DATABASE_PIMP_KEGG)

2020-01-06 11:22:54.241 | DEBUG    | pals.feature_extraction:__init__:38 - Using PiMP_KEGG as database
2020-01-06 11:22:54.242 | DEBUG    | pals.feature_extraction:get_database:101 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2020-01-06 11:22:54.264 | DEBUG    | pals.feature_extraction:__init__:51 - Mapping pathway to unique ids
2020-01-06 11:22:54.269 | DEBUG    | pals.feature_extraction:__init__:65 - Creating dataset to pathway mapping
2020-01-06 11:22:54.278 | DEBUG    | pals.feature_extraction:__init__:93 - Computing unique id counts


### PALS analysis

In [13]:
pals = PALS(crc_ds, min_replace=5000, plage_weight=5, hg_weight=1)
pathway_df = pals.get_pathway_df()

2020-01-06 11:22:54.358 | DEBUG    | pals.pathway_analysis:_change_zero_peak_ints:407 - Setting the zero intensity values in the dataframe
2020-01-06 11:22:54.421 | DEBUG    | pals.pathway_analysis:_standardize_intensity_df:388 - Scaling the data across the sample: zero mean and unit variance
2020-01-06 11:22:54.424 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:215 - Mean values of the rows in the DF is [-0. -0. -0.  0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0.  0.  0.  0. -0.
 -0.  0.  0. -0. -0.  0. -0.  0. -0. -0. -0.  0. -0.  0.  0.  0. -0. -0.
  0.  0.  0. -0. -0. -0. -0.  0.  0. -0. -0. -0.  0. -0.  0. -0.  0.  0.
  0. -0.  0. -0.  0.  0. -0. -0. -0. -0.  0. -0. -0.  0.  0.  0. -0.  0.
 -0. -0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0.  0.
  0.  0. -0.  0. -0. -0. -0. -0. -0. -0.  0. -0.  0. -0. -0.]
2020-01-06 11:22:54.426 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:216 - Variance in the rows of the DF is [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [14]:
pd.set_option('display.max_rows', None)

In [15]:
pathway_df.sort_values('PiMP_KEGG crc/healthy comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,crc/healthy p-value,polyp/healthy p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,PiMP_KEGG crc/healthy comb_p,PiMP_KEGG polyp/healthy comb_p
map00410,beta-Alanine metabolism,0.000362,1.000000,31,9,29.03,2.107099e-07,0.97,3.13,0.000008,0.999999
map00340,Histidine metabolism,0.000087,1.000000,41,6,14.63,1.439225e-03,1.28,3.12,0.000010,1.000000
map00660,C5-Branched dibasic acid metabolism,0.000747,1.000000,21,8,38.10,9.426139e-08,0.65,3.10,0.000018,1.000000
map00240,Pyrimidine metabolism,0.001296,0.972813,56,13,23.21,6.773139e-09,1.74,3.11,0.000024,0.780160
map00300,Lysine biosynthesis,0.000228,0.999284,27,5,18.52,1.228408e-03,0.84,3.11,0.000028,0.994335
map04724,Glutamatergic synapse,0.000567,1.000000,7,2,28.57,1.818771e-02,0.22,3.14,0.000158,1.000000
map00910,Nitrogen metabolism,0.000567,1.000000,17,2,11.76,9.639169e-02,0.53,3.12,0.000283,1.000000
map00250,"Alanine, aspartate and glutamate metabolism",0.016520,1.000000,23,11,47.83,1.424652e-11,0.72,3.13,0.000343,0.999979
map00121,Secondary bile acid biosynthesis,0.001637,1.000000,9,2,22.22,2.994154e-02,0.28,3.11,0.000572,1.000000
map04964,Proximal tubule bicarbonate reclamation,0.010933,1.000000,16,7,43.75,2.042991e-07,0.50,3.12,0.000595,1.000000


### ORA Analysis

In [16]:
ds = DataSource(crc_int_df, crc_annotation_df, crc_experimental_design, DATABASE_PIMP_KEGG)

2020-01-06 11:23:02.620 | DEBUG    | pals.feature_extraction:__init__:38 - Using PiMP_KEGG as database
2020-01-06 11:23:02.621 | DEBUG    | pals.feature_extraction:get_database:101 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2020-01-06 11:23:02.645 | DEBUG    | pals.feature_extraction:__init__:51 - Mapping pathway to unique ids
2020-01-06 11:23:02.652 | DEBUG    | pals.feature_extraction:__init__:65 - Creating dataset to pathway mapping
2020-01-06 11:23:02.664 | DEBUG    | pals.feature_extraction:__init__:93 - Computing unique id counts


In [17]:
pals = PALS(ds, min_replace=5000)
pathway_df = pals.get_ora_df()

2020-01-06 11:23:02.739 | DEBUG    | pals.pathway_analysis:get_ora_df:50 - Calculating ORA
2020-01-06 11:23:02.740 | DEBUG    | pals.pathway_analysis:_change_zero_peak_ints:407 - Setting the zero intensity values in the dataframe
2020-01-06 11:23:04.173 | DEBUG    | pals.pathway_analysis:get_ora_df:111 - Correcting for multiple t-tests
2020-01-06 11:23:04.179 | DEBUG    | pals.pathway_analysis:_calculate_coverage_df:493 - Calculating dataset formula coverage


In [18]:
pathway_df.sort_values('PiMP_KEGG crc/healthy comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,crc/healthy p-value,polyp/healthy p-value,PiMP_KEGG crc/healthy comb_p,PiMP_KEGG polyp/healthy comb_p,unq_pw_F,tot_ds_F,F_coverage
mapids,,,,,,,,
map00970,Aminoacyl-tRNA biosynthesis,5.536095e-11,1.000000,8.304143e-09,1.000000,23,18,78.26
map00250,"Alanine, aspartate and glutamate metabolism",3.002766e-09,1.000000,2.252074e-07,1.000000,23,11,47.83
map02010,ABC transporters,8.094736e-09,0.010530,3.035526e-07,0.225644,79,27,34.18
map00330,Arginine and proline metabolism,8.094736e-09,0.154929,3.035526e-07,0.663983,79,19,24.05
map04974,Protein digestion and absorption,1.147609e-08,1.000000,3.442826e-07,1.000000,42,20,47.62
map00410,beta-Alanine metabolism,3.020148e-08,1.000000,7.550370e-07,1.000000,31,9,29.03
map04964,Proximal tubule bicarbonate reclamation,5.981264e-07,0.033152,1.281699e-05,0.320417,16,7,43.75
map02020,Two-component system,1.574146e-06,0.069348,2.951523e-05,0.358696,34,10,29.41
map04727,GABAergic synapse,1.784262e-06,1.000000,2.973769e-05,1.000000,9,4,44.44
